In [1]:
%reload_ext ishbook
import pandas as pd
import numpy as np
import datetime as dt
import time
import iql
import plus
import re
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Pull in the affected Advertisers from IQL query provided by CSP
* Elaborate on the advid to get the advertiser name

In [3]:
%%ish

tcpa_complaint = from advertiserevent 2016-10-01 today where service_rep_id = 2 activity_type IN (CONTACT_WEB_FORM, ZENDESK_ESCALATION, LEAD_FROM_HIRE_CONTACT_FORM) group by advertiser_id, service_rep_username
Output = None

In [4]:
tcpa_complaint.advertiser_id.nunique()

4832

In [5]:
okay_to_contact = plus.get_advertiser_info(tcpa_complaint.advertiser_id)

LDAP name:
dmoncada
Password:
········


In [10]:
print okay_to_contact.showHostedJobs.value_counts()
print okay_to_contact.active.value_counts()

verified                1864
enabled                 1381
rejected                1010
verified_spononly        244
pending                  174
disabled                 157
more_information_req       2
Name: showHostedJobs, dtype: int64
2    2707
1    1636
0     489
Name: active, dtype: int64


In [13]:
okay_to_contact.head()['bid_managed']

advertiser_id
29775    0
38902    0
56241    0
60412    1
60646    0
Name: bid_managed, dtype: object

In [12]:
okay_to_contact.columns

Index([u'account_email', u'account_id', u'active', u'agency_city',
       u'agency_country', u'agency_id', u'agency_location_id', u'agency_name',
       u'agency_rep', u'agency_state', u'bid_managed', u'billing_country',
       u'billing_state', u'billing_zip', u'commission_last_date',
       u'commission_start_date', u'company', u'currency', u'date_created',
       u'employee_count', u'first_revenue_date', u'industry', u'is_ad_agency',
       u'last_revenue_date', u'locale', u'masteraccount_id', u'okay_to_call',
       u'parent_company_id', u'sales_rep', u'service_rep', u'showHostedJobs',
       u'strategic_rep', u'type'],
      dtype='object')

In [14]:
okay_to_contact_2 = okay_to_contact[['last_revenue_date', 'active', 'okay_to_call', 'showHostedJobs']]

In [17]:
combined_df = tcpa_complaint.merge(okay_to_contact_2, left_on = "advertiser_id", right_index = True)

In [22]:
active = combined_df['active'] > 0
jobstatus = combined_df['showHostedJobs'].isin(['verified', 'pending', 'enabled', 'verified_spononly'])
current_rev = combined_df['last_revenue_date'] > '2016-01-01'

In [23]:
final_df = combined_df[active & jobstatus & current_rev]

In [24]:
len(final_df)

33500

In [30]:
final_df.okay_to_call.value_counts()

True     17598
False    15902
Name: okay_to_call, dtype: int64